# Importing and loading data

In [60]:
import pandas as pd
import seaborn as sns
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt

df = pd.read_csv('20200124_ews_observations.csv', low_memory=False)
df = df.dropna()
df['ObservationDate'] = pd.to_datetime(df['ObservationDate'])

**Function that deletes the unnecessary columns so that only columns with numbers are kept and the datetime**

In [61]:
def remove_scores(df):
    df = df.drop(['SBP_Score', 'SpO2_Score', 'HR_Score', 'TEMP_Score', 'RR_Score', 'EwsProcedure', 'Add_O2', 'LOC'], axis=1)
    return df

**Function that returns a dataframe of the patient and some info about the measurements**

In [90]:
def analyse_patient (df, id_nr, plot=False):
    # only select rows from the patient
    patient = df.loc[df['PatientId'] == id_nr]
    patient = patient.sort_values(by='ObservationDate', ascending=True)
    # datetime as index
    patient = patient.set_index('ObservationDate')
    # drop NAN values and check how many samples are useful
    patient = patient.dropna()
    n_measurements = patient.shape[0]
    print(f"Number of measurements: {n_measurements}")
    
    patient = patient.drop(['EwsProcedure'], axis=1)
    
    # print the info
    if n_measurements > 1:
        # print average minimum and maximum time between measurements
        delta_t = patient.index.to_series().diff().dt.seconds.div(3600, fill_value=0)
        delta_t_avg = delta_t.mean()
        delta_t_max = delta_t.max()
        delta_t_min = delta_t.nsmallest(2).iloc[1]
        print(f"Average time (hours) between measurements of patient {id_nr}: {delta_t_avg}")
        print(f"Max diff time between measurements: {delta_t_max}")
        print(f"Min diff time between measurements: {delta_t_min}")
        
        # print start and end date of hospitalisation
        start_date = patient.index.values[0]
        end_date = patient.index.values[-1]
        print(f"Start_date: {start_date}")
        print(f"End_date: {end_date}")
        
        # check how many data we have on the patient
        delta_t = end_date - start_date
        delta_days = delta_t.astype('timedelta64[D]').astype('str')
        # delta_days prints "xx days"
        # next line cuts " days" out of the string delta_days
        delta_days = delta_days[:len(delta_days) - 5]
        print(f"Hospitalisation duration: {delta_days}")
        
    else:
        print("Only 1 measurement")
    
    
    
#     if int(delta_days) >= 2 and n_measurements >= 15:
#         to_predict = True
#         print("This patient has enough data to apply an accurate model")
#     else:
#         to_predict = False
#         print("Not enough data yet to predict EWS")
    
    # plot timeseries
    if plot:
        fig, ax = plt.subplots(figsize=(15,15))
        sns.lineplot(data=patient, x="ObservationDate", y='EWS_Total', linestyle='-', color='grey', ax=ax)
        sns.scatterplot(data=patient, x="ObservationDate", y='EWS_Total', marker='o', linestyle='-', hue="Add_O2_Score", palette="tab10", ax=ax)
    
    return patient
#    return to_predict

In [100]:
patientx = analyse_patient(df, 1743)
patientx.head()

Number of measurements: 616
Average time (hours) between measurements of patient 1743: 7.080062680375186
Max diff time between measurements: 23.8975
Min diff time between measurements: 0.0022222222222222222
Start_date: 2019-06-15T13:32:39.000000000
End_date: 2020-01-02T06:51:46.000000000
Hospitalisation duration: 200


,PatientId,EWS_Total,SBP,SBP_Score,LOC,LOC_Score,SpO2,SpO2_Score,Add_O2,Add_O2_Score,HR,HR_Score,RR,RR_Score,TEMP,TEMP_Score
ObservationDate,,,,,,,,,,,,,,,,
2019-06-15 13:32:39,1743,4,122.0,0.0,ALERT,0,91.0,0.0,True,2.0,112,2,18,0,36.6,0
2019-06-15 20:22:36,1743,5,109.0,1.0,ALERT,0,93.0,1.0,True,2.0,104,1,16,0,37.0,0
2019-06-16 14:23:34,1743,3,117.0,0.0,ALERT,0,89.0,0.0,True,2.0,103,1,19,0,36.5,0
2019-06-16 19:49:24,1743,3,123.0,0.0,ALERT,0,90.0,0.0,True,2.0,107,1,19,0,36.6,0
2019-06-17 04:39:15,1743,4,102.0,1.0,ALERT,0,92.0,0.0,True,2.0,97,1,18,0,36.7,0


**Function that returns a dictionary of the d values of the metrics**

In [101]:
def n_diffs(df_patient):
    from pmdarima.arima.utils import ndiffs
    n_diffs_dict  = {}
    metrics = ['SBP', 'SpO2', 'HR', 'RR', 'TEMP']
    for column in metrics:
        y = df_patient[column]
        n_diffs = ndiffs(y, test='adf')
        n_diffs_dict[column] = n_diffs
    return n_diffs_dict

**Function that returns a dictionary of the parameters of the models for each of the metrics**

In [102]:
def get_best_pdq(df_patient):
    from pmdarima.arima import auto_arima
    metric_pdq_dict = {}
    dict_ndiffs = n_diffs(df_patient)
    for metric in dict_ndiffs:
        ndiffs = dict_ndiffs[metric]
        best_pdq = auto_arima(df_patient[metric], d=ndiffs, trace=False)
        metric_pdq_dict[metric] = best_pdq.order
    return metric_pdq_dict

In [103]:
# from sklearn.metrics import mean_squared_error
# def get_best_pdq(df_patient):
#     from pmdarima.arima import auto_arima
#     metric_pdq_dict = {}
#     dict_ndiffs = n_diffs(df_patient)
#     for metric in dict_ndiffs:
#         X = df_patient[metric].values
#         size = int(len(X) * 0.66)
#         train, test = X[0:size], X[size:len(X)]
#         ndiffs = dict_ndiffs[metric]
#         model = auto_arima(train, d=ndiffs, trace=False)
#         preds, conf_int = model.predict(n_periods=test.shape[0], return_conf_int=True)
#         print(f"Test RMSE {metric}: {np.sqrt(mean_squared_error(test, preds))}" )
#     return metric_pdq_dict
# get_best_pdq(patientx)

**Function that models and fits an ARIMA model for each of the metrics and also plots things**

In [106]:
def model_and_predict(df_patient):
    from statsmodels.tsa.arima_model import ARIMA
    from sklearn.model_selection import TimeSeriesSplit
    
    metrics = ['SBP', 'SpO2', 'HR', 'RR', 'TEMP']
    pdq_dict = get_best_pdq(patientx)
    print(pdq_dict)
    for metric in metrics:
        X = df_patient[metric].values
        size = int(len(X) * 0.66)
        train, test = X[0:size], X[size:len(X)]
        history = [x for x in train]
        predictions = []
        df_patient[metric]
        print(f"-----------------------------{metric}--------------------------------")
        for t in range(len(test)): 
            model = ARIMA(history, order=(pdq_dict[metric]))
            model_fit = model.predict(start_params = [1, 5], disp=-1, transparams=False)
            output = model_fit.forecast()
            yhat = output[0]
            predictions.append(yhat)
            obs = test[t]
            history.append(obs)
            #print('predicted=%f, expected=%f' % (yhat, obs))
        
        print(model_fit.summary())
        

In [107]:
import warnings
warnings.filterwarnings('ignore')

model_and_predict(patientx)

{'SBP': (5, 0, 5), 'SpO2': (1, 0, 5), 'HR': (3, 0, 0), 'RR': (3, 0, 3), 'TEMP': (3, 0, 0)}
-----------------------------SBP--------------------------------


ValueError: could not broadcast input array from shape (0,1) into shape (5,1)

**Functions that converts scores to EWS Total score**  
Two scales for SpO2

In [8]:
def calculate_news2_scale1 (rr, spo2, air_score, sbp, hr, loc_score, temp, check=False):
    score = 0
    # RR SCORE - normal values: 12-20
    if rr <= 8 or rr >= 25:
        rr_score = 3
    elif 21 <= rr <= 24:
        rr_score = 2
    elif 9 <= rr <= 11:
        rr_score = 1
    else:
        rr_score = 0
    
    # SPO2 SCORE - normal values: >= 96
    if spo2 <= 91:
        spo2_score = 3
    elif 92 <= spo2 <= 93:
        spo2_score = 2
    elif 94 <= spo2 <= 95:
        spo2_score = 1
    else:
        spo2_score = 0
    
    # OXYGEN METHOD SCORE 
    # 0 -> Room Air 
    # 3 -> Oxygen Mask
    try:
        assert air_score == 0 or air_score == 2, "Air_score is not 0 or 2"
    except AssertionError:
        raise
    
    # SBP SCORE - normal values: 111-219
    if sbp <= 90 or sbp >= 220:
        sbp_score = 3
    elif 91 <= sbp <= 100:
        sbp_score = 2
    elif 101 <= sbp <= 110:
        sbp_score = 1
    else:
        sbp_score = 0
    
    # HR SCORE - normal values: 51-90
    if hr <= 40 or hr >= 131:
        hr_score = 3
    elif 111 <= hr <= 130:
        hr_score = 2
    elif 91 <= hr <= 110 or 41 <= hr <= 50:
        hr_score = 1
    else:
        hr_score = 0
    
    # CONSCIOUSNESS METHOD SCORE 
    # 0 -> Alert 
    # 3 -> CVPU (Confusion, Voice, Pain, Unresponsive)
    try:
        assert loc_score == 0 or loc_score == 3, "loc_score is not 0 or 3"
    except AssertionError:
        raise
    
    # TEMP SCORE - normal values: 51-90
    if temp <= 35.0:
        temp_score = 3
    elif temp > 39.1:
        temp_score = 2
    elif 35.1 <= temp <= 36.0 or 38.1 <= temp <= 39.0:
        temp_score = 1
    else:
        temp_score = 0
    
    if check:
        print(f'rr_score: {rr_score}')
        print(f'spo2_score: {spo2_score}')
        print(f'air_score: {air_score}')
        print(f'sbp_score: {sbp_score}')
        print(f'hr_score: {hr_score}')
        print(f'loc_score: {loc_score}')
        print(f'temp_score: {temp_score}')
    
    ews_total = rr_score + spo2_score + air_score + sbp_score + hr_score + loc_score + temp_score
    
    return ews_total

In [9]:
def calculate_news2_scale2 (rr, spo2, air_score, sbp, hr, loc_score, temp, check=False):
    score = 0
    # RR SCORE - normal values: 12-20
    if rr <= 8 or rr >= 25:
        rr_score = 3
    elif 21 <= rr <= 24:
        rr_score = 2
    elif 9 <= rr <= 11:
        rr_score = 1
    else:
        rr_score = 0
    
    # SPO2 SCORE - normal values: >= 96
    if spo2 <= 83 or spo2 >= 97:
        spo2_score = 3
    elif 84 <= spo2 <= 85 or 95 <= spo2 <= 96:
        spo2_score = 2
    elif 86 <= spo2 <= 87 or 93 <= spo2 <= 94:
        spo2_score = 1
    else:
        spo2_score = 0
    
    # OXYGEN METHOD SCORE 
    # 0 -> Room Air 
    # 3 -> Oxygen Mask
    try:
        assert air_score == 0 or air_score == 2, "Air_score is not 0 or 2"
    except AssertionError:
        raise
    
    # SBP SCORE - normal values: 111-219
    if sbp <= 90 or sbp >= 220:
        sbp_score = 3
    elif 91 <= sbp <= 100:
        sbp_score = 2
    elif 101 <= sbp <= 110:
        sbp_score = 1
    else:
        sbp_score = 0
    
    # HR SCORE - normal values: 51-90
    if hr <= 40 or hr >= 131:
        hr_score = 3
    elif 111 <= hr <= 130:
        hr_score = 2
    elif 91 <= hr <= 110 or 41 <= hr <= 50:
        hr_score = 1
    else:
        hr_score = 0
    
    # CONSCIOUSNESS METHOD SCORE 
    # 0 -> Alert 
    # 3 -> CVPU (Confusion, Voice, Pain, Unresponsive)
    try:
        assert loc_score == 0 or loc_score == 3, "loc_score is not 0 or 3"
    except AssertionError:
        raise
    
    # TEMP SCORE - normal values: 51-90
    if temp <= 35.0:
        temp_score = 3
    elif temp > 39.1:
        temp_score = 2
    elif 35.1 <= temp <= 36.0 or 38.1 <= temp <= 39.0:
        temp_score = 1
    else:
        temp_score = 0
    
    if check:
        print(f'rr_score: {rr_score}')
        print(f'spo2_score: {spo2_score}')
        print(f'air_score: {air_score}')
        print(f'sbp_score: {sbp_score}')
        print(f'hr_score: {hr_score}')
        print(f'loc_score: {loc_score}')
        print(f'temp_score: {temp_score}')
    
    ews_total = rr_score + spo2_score + air_score + sbp_score + hr_score + loc_score + temp_score
    
    return ews_total

In [10]:
def check_hit_acc (df, samples, to_print=False):
# if there is a hit with either of the scales it will append true to this list
    check_row = []
    test_df = df.sample(n=samples)
    # check every row and apply calculate_news2_scale1
    for index, row in test_df.iterrows():
        ews_row_calc = calculate_news2_scale1(rr=row.RR, spo2=row.SpO2, air_score=row.Add_O2_Score, sbp=row.SBP, 
                                       hr=row.HR, loc_score=row.LOC_Score, temp=row.TEMP, check=to_print)
        if int(ews_row_calc) == int(row.EWS_Total):
            check_row.append(True)
            if to_print == True:
                print("-----")
        # if this isn't a hit, apply calculate_news2_scale2 
        else:
            ews_row_calc_2 = calculate_news2_scale2(rr=row.RR, spo2=row.SpO2, air_score=row.Add_O2_Score, sbp=row.SBP, 
                                       hr=row.HR, loc_score=row.LOC_Score, temp=row.TEMP, check=to_print)
            if ews_row_calc_2 == int(row.EWS_Total):
                check_row.append(True)
                if to_print == True:
                    print("-----")
            # if both of the calculate methods fail, false will be appended to the list
            else:
                check_row.append(False)
                if to_print == True:
                    print(f"{row.name}: false")
                    print(f"calculated: {ews_row_calc} != given: {row.EWS_Total}")
                    print("-----")
    # print the accuracy of this function
    hit_perc = (check_row.count(True)/len(check_row))*100
    print(f"Accuracy of function: {hit_perc}")

In [11]:
hit_acc_50 = check_hit_acc(df, samples=5000)

Accuracy of function: 96.64


In [12]:
n_diffs(df)

{'SBP': 0, 'SpO2': 0, 'HR': 0, 'RR': 0, 'TEMP': 0}